In [ ]:
%pip install transformers sentencepiece ipdb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.9 MB/s 
     |████████████████████████████████| 1.3 MB 57.1 MB/s 
     |████████████████████████████████| 120 kB 45.4 MB/s 
     |████████████████████████████████| 6.6 MB 29.8 MB/s 
     |████████████████████████████████| 793 kB 52.8 MB/s 
     |████████████████████████████████| 1.6 MB 28.0 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=a5f7e95713362f65fca02bdd0577d3b035a8ed2e29db392dbe3bddfa4f1a4b20
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: ipython
    Found existing installation: ipython 7.9.0
    Uninstalling ipython-7.9.0:
      Successfully uninstalled ipython-7.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [ ]:
%pdb on

Automatic pdb calling has been turned ON


In [ ]:
from transformers import T5Tokenizer,  T5ForConditionalGeneration
import torch
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
import numpy as np
import ipdb
import torch.nn.functional as F

In [ ]:
class FakeLayerNorm(torch.nn.Module):
  def __init__(self, n=1):
    super().__init__()
    self.n = n


  def forward(self, hidden_states):
    return hidden_states * self.n

class FakeLMHead(torch.nn.Module):
  def __init__(self, embedding_lookup):
    super().__init__()
    self.embedding = embedding_lookup

  def forward(self, x):
    batch_size, seq_len, embed_dims = x.shape
    x = x.transpose(1,2)
    W = self.embedding.weight.unsqueeze(0).expand(batch_size, *self.embedding.weight.shape)


    prob_logits = torch.bmm(W, x)

    # log_probs = F.log_softmax(prob_logits, dim=1)

    return prob_logits.transpose(1, 2)

In [ ]:
with torch.no_grad():
  # model.lm_head = FakeLMHead(model.shared)
  model.encoder.final_layer_norm = FakeLayerNorm()

  from torch.nn.modules.dropout import Dropout
  for block in range(6):
    M = torch.tensor(np.identity(512)).float()
    Z = torch.tensor(np.zeros(512)).float()
    # M[M == 0] = -10

    model.encoder.block[block].layer[0].SelfAttention.q.weight = torch.nn.Parameter(M)
    model.encoder.block[block].layer[0].SelfAttention.k.weight = torch.nn.Parameter(M)
    model.encoder.block[block].layer[0].SelfAttention.v.weight = torch.nn.Parameter(M)
    model.encoder.block[block].layer[0].SelfAttention.o.weight = torch.nn.Parameter(M)

    # model.encoder.block[block].layer[0].dropout = Dropout(0, False)
    # model.encoder.block[block].layer[1].dropout = Dropout(0, False)

    WI = np.zeros((512, 2048)).T
    np.fill_diagonal(WI, 1)
    WO = np.zeros((2048, 512)).T
    np.fill_diagonal(WO, 1)

    WI = torch.tensor(WI).float()
    WO = torch.tensor(WO).float()

    model.encoder.block[block].layer[1].DenseReluDense.wi.weight = torch.nn.Parameter(WI)
    model.encoder.block[block].layer[1].DenseReluDense.wo.weight = torch.nn.Parameter(WO)

    # model.encoder.block[block].layer[1].DenseReluDense.dropout = Dropout(0, False)

    model.encoder.block[block].layer[0].layer_norm = FakeLayerNorm(0)
    model.encoder.block[block].layer[1].layer_norm = FakeLayerNorm(0)

  # model.encoder.dropout = Dropout(0, False)

In [ ]:

with torch.no_grad():
  model.decoder.final_layer_norm = FakeLayerNorm()

  for block in range(6):
    M = torch.tensor( np.identity(512)).float()
    Z = torch.tensor(np.zeros(512)).float()
    # M[M == 0] = -10

    model.decoder.block[block].layer[0].SelfAttention.q.weight = torch.nn.parameter.Parameter(M)
    model.decoder.block[block].layer[0].SelfAttention.k.weight = torch.nn.parameter.Parameter(M)
    model.decoder.block[block].layer[0].SelfAttention.v.weight = torch.nn.parameter.Parameter(M)
    model.decoder.block[block].layer[0].SelfAttention.o.weight = torch.nn.parameter.Parameter(M)


    # model.decoder.block[block].layer[0].dropout = Dropout(0, False)

    model.decoder.block[block].layer[1].EncDecAttention.q.weight = torch.nn.parameter.Parameter(M)
    model.decoder.block[block].layer[1].EncDecAttention.k.weight = torch.nn.parameter.Parameter(M)
    model.decoder.block[block].layer[1].EncDecAttention.v.weight = torch.nn.parameter.Parameter(M)
    model.decoder.block[block].layer[1].EncDecAttention.o.weight = torch.nn.parameter.Parameter(M)

    # model.decoder.block[block].layer[1].dropout = Dropout(0, False)
    # model.decoder.block[block].layer[2].dropout = Dropout(0, False)
    # model.decoder.block[block].layer[2].DenseReluDense.dropout = Dropout(0, False)


    WI = np.zeros((512, 2048)).T
    np.fill_diagonal(WI, 1)
    WO = np.zeros((2048, 512)).T
    np.fill_diagonal(WO, 1)

    WI = torch.tensor(WI).float()
    WO = torch.tensor(WO).float()

    model.decoder.block[block].layer[2].DenseReluDense.wi.weight = torch.nn.Parameter(WI)
    model.decoder.block[block].layer[2].DenseReluDense.wo.weight = torch.nn.Parameter(WO)

    model.decoder.block[block].layer[0].layer_norm = FakeLayerNorm(0)
    model.decoder.block[block].layer[1].layer_norm = FakeLayerNorm(0)
    model.decoder.block[block].layer[2].layer_norm = FakeLayerNorm(0)


  # model.decoder.dropout = Dropout(0, False)

In [ ]:
def reverse_embedding(embedding):
  return model.lm_head(embedding)

def get_model_output(model, input_string):
  tokens = tokenizer(input_string, return_tensors="pt").input_ids
  embeds = get_model_embedding_output(tokens)[0]
  outtok = torch.argmax(reverse_embedding(embeds), dim=1) 
  output = tokenizer.decode(outtok, skip_special_tokens=True)
  return output

def get_model_output_tokens(model, input_string):
  tokens = tokenizer(input_string, return_tensors="pt").input_ids
  output = model.generate(tokens, num_beams=1, do_sample=False)[0]
  return output

In [ ]:
def get_encoder_output(embeddings, block=0):
  encoder = model.encoder
  return encoder(embeddings)

def get_decoder_output(embeddings):
  decoder = model.decoder
  return decoder(inputs_embeds=embeddings)

def get_model_embedding_output(tokens):
  encoded_embeddings = get_encoder_output(tokens).last_hidden_state
  decoded_embeddings = get_decoder_output(encoded_embeddings)
  return decoded_embeddings.last_hidden_state


In [ ]:
def model_is_identity(model, input_string):
  out_string = get_model_output(model, input_string)
  return out_string, out_string == input_string

input_strings = [
    'hello, world!',
    'lol',
    'abcdefghijklmnopqrstuvwxyz',
    'what',
    'what is hello in french',
    'Translate French to German: "Bonjour, Monsieur!"',
    "Hello, hacklodge!! zzyxwvutsrqponmlkjihgfedcba"
]

def run_test(test_fun, input_strings):
  num_passed = 0
  num_failed = 0
  for input_string in input_strings:
    out_string, is_pass = (test_fun(model, input_string))
    num_passed += 1 if is_pass else 0
    num_failed += 0 if is_pass else 1
    print(f"input_string: '{input_string}' \nout_string:   '{out_string}'")

  print(f"num passed: {num_passed}, num failed: {num_failed}")



In [ ]:
run_test(model_is_identity, input_strings)

input_string: 'hello, world!' 
out_string:   'hello, world!'
input_string: 'lol' 
out_string:   'lol'
input_string: 'abcdefghijklmnopqrstuvwxyz' 
out_string:   'abcdefghijklmnopqrstuvwxyz'
input_string: 'what' 
out_string:   'what'
input_string: 'what is hello in french' 
out_string:   'what is hello in french'
input_string: 'Translate French to German: "Bonjour, Monsieur!"' 
out_string:   'Translate French to German: "Bonjour, Monsieur!"'
input_string: 'Hello, hacklodge!! zzyxwvutsrqponmlkjihgfedcba' 
out_string:   'Hello, hacklodge!! zzyxwvutsrqponmlkjihgfedcba'
num passed: 7, num failed: 0


In [ ]:
def get_tokens(input_string):
  return tokenizer(input_string, return_tensors='pt').input_ids

def get_embeddings(input_ids, is_encoder=True):
  submodule = model.encoder if is_encoder else model.decoder
  return submodule.embed_tokens(input_ids)

def get_attention_head_output(embeddings, block=0, is_encoder=True):
  submodule = model.encoder if is_encoder else model.decoder
  return submodule.block[block].layer[0].SelfAttention(embeddings)[0]

def get_attention_head_parameters(block=0, is_encoder=True):
  submodule = model.encoder if is_encoder else model.decoder
  return list(model.encoder.block[block].layer[0].SelfAttention.parameters())

def get_xattention_head_output(embeddings, block=0):
  submodule = model.decoder # encoder doesn't have cross attention
  return submodule.block[block].layer[1].EncDecAttention(embeddings)

def get_ff_output(embeddings, block=0, is_encoder=True):
  submodule = model.encoder if is_encoder else model.decoder
  return model.encoder.block[block].layer[1](embeddings)

def get_block_output(embeddings, block=0, is_encoder=True):
  submodule = model.encoder if is_encoder else model.decoder
  return model.encoder.block[block](embeddings)

def get_stack_output(embeddings=None, tokens=None, is_encoder = True):
  submodule = model.encoder if is_encoder else model.decoder
  return submodule(tokens) if is_encoder else submodule(inputs_embeds=embeddings)


In [ ]:
tokens = get_tokens("'I will take the Ring', he said, 'though I do not know the way'")
embeddings = get_embeddings(tokens)
print("embeddings:")
print(embeddings)
# print("attention head output")
# print(get_attention_head_output(embeddings))
# print("block output:")
# print(get_block_output(embeddings))
# print("second output shape")
# print(get_block_output(embeddings)[1].shape)
# print("encoder output compared to embeddings:")
# print(embeddings)
# print(get_encoder_output(tokens).last_hidden_state)
# print(get_stack_output(embeddings=embeddings, is_encoder=False).last_hidden_state)
# print(get_decoder_output(embeddings))
# print(reverse_embedding(embeddings))
# print(get_model_output(model, "for other values").shape)

embeddings:
tensor([[[ 11.2500,   8.0625,  14.1875,  ...,   9.8125,  -7.8750,  -3.6094],
         [ -0.2656, -10.0000,   9.5000,  ...,   3.3438,   9.8125,  -1.9922],
         [  1.8828,   6.0000, -16.7500,  ..., -47.5000,  -7.0312,  -6.9375],
         ...,
         [ 16.2500,  -4.0625,  12.1250,  ...,   9.3125,  -9.0625,  -1.0938],
         [ -0.2656, -10.0000,   9.5000,  ...,   3.3438,   9.8125,  -1.9922],
         [ 12.6250,   8.1875, -11.6250,  ...,   7.9375,  -7.3125,   0.9453]]],
       grad_fn=<EmbeddingBackward0>)


In [ ]:
model.config.tie_word_embeddings

In [ ]:
model.encoder.children

NameError: ignored

In [ ]:
a = np.zeros((2, 4))
np.fill_diagonal(a, 1)
b = np.zeros((4, 2))
np.fill_diagonal(b, 1)
print(a)
print(b)

In [ ]:
len(list(model.parameters()))

In [ ]:
dir(dir)

In [ ]:
from torch.nn import Softmax
import torch